# Problem 1

In [9]:
import math

In [6]:
# convert data from txt file to dict
dicts = {}
i = 0
with open("W2V_150.txt") as f:
    for lines in f:
        i += 1
        if i >= 3:
            line = lines.split()
            key = line[0]
            line.pop(0)
            val = line
            dicts[str(key)] = val
            for i in range(0, len(val)):
                val[i] = float(val[i])

In [5]:
def cosine_similarity(v1,v2):
    xx, xy, yy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        xx += x*x
        yy += y*y
        xy += x*y
    return xy/math.sqrt(xx*yy)

In [25]:
from prettytable import PrettyTable
table = PrettyTable(['Word 1', 'Word 2', 'Cosine_similarity'])

In [26]:
from scipy import stats
i = 0
with open("ViSim-400.txt") as f:
    for lines in f:
        i += 1
        if i > 1:
            line = lines.split()
            if line[0] in dicts.keys() and line[1] in dicts.keys():
                v1 = dicts.get(line[0])        
                v2 = dicts.get(line[1])
                table.add_row([line[0], line[1], cosine_similarity(v1, v2)])
            else:
                table.add_row([line[0], line[1], 'NaN'])
print(table)

+---------------------+---------------------+------------------------+
|        Word 1       |        Word 2       |   Cosine_similarity    |
+---------------------+---------------------+------------------------+
|         biến        |         ngập        | -0.0049123394696700545 |
|     nhà_thi_đấu     |         nhà         |  0.08252318329211768   |
|         động        |         tĩnh        |   0.2770859598682775   |
|        khuyết       |          ưu         |  0.17679862835626708   |
|       cõi_tục       |        cõi_âm       |          NaN           |
|       thủ_pháp      |      biện_pháp      |  0.40236612919430603   |
|      kết_duyên      |      thành_hôn      |   0.4630084020140725   |
|       cấp_tiến      |       bảo_thủ       |   0.2569470088996125   |
|       nước_lớn      |      nguy_hiểm      |   0.1851920224021162   |
|         hoa         |         nhị         |  0.22306960452548538   |
|       bất_lợi       |      thuận_lợi      |   0.5348913029963387   |
|     

In [264]:
#convert list from old range value to new range value
import numpy as np

def convert_between_range(sim, old_range, new_range):
    old_min = old_range[0]
    old_max = old_range[1]
    new_min = new_range[0]
    new_max = new_range[1]
    new_sim = []
    for s in sim:
        if s == 'NaN':
            continue
        else:
            temp_s = new_min + (s-old_min)/(old_max-old_min)*(new_max-new_min)
            new_sim.append(temp_s)
    return new_sim

In [265]:
from scipy import stats

my_sim_1 = convert_between_range(my_sim, [-1, 1], [0, 6])
my_sim_2 = convert_between_range(my_sim, [-1, 1], [0, 10])

print("Pearson correlation coefficient between sim_1 and cosine similarity: ", stats.pearsonr(sim_1,my_sim_1))
print("Pearson correlation coefficient between sim_2 and cosine similarity: ", stats.pearsonr(sim_2,my_sim_2))
print("Spearman's rank correlation coefficient sim_1 and cosine similarity: ", stats.spearmanr(sim_1,my_sim_1))
print(" Spearman's rank correlation coefficient sim_2 and cosine similarity: ", stats.spearmanr(sim_2,my_sim_2))

NameError: name 'my_sim' is not defined

# Problem 2

In [35]:
import operator

# Example k = 10, w = 'tiết_lộ'
k = int(input())
word = input()
sim_mp = {}
if word in dicts.keys():
    for key in dicts:
        if (word != key):
            sim_mp[key] = cosine_similarity(dicts.get(word), dicts.get(key))
    sim_mp = sorted(sim_mp.items(), key=operator.itemgetter(1), reverse=True)
    key_list = list(sim_mp)
    for i in range(k):
        print(key_list[i])

10
tiết_lộ
('hé_lộ', 0.8141896754726131)
('bật_mí', 0.6960303061638391)
('công_bố', 0.5812819713581816)
('xác_nhận', 0.5785359220947899)
('thổ_lộ', 0.5612686986844789)
('phát_hiện_ra', 0.5585788908739554)
('loan_báo', 0.5559826694440024)
('khám_phá_ra', 0.5496182853839372)
('thú_nhận', 0.5479432813714646)
('úp_mở', 0.5438636450711214)


# Problem 3

In [47]:
import numpy as np

In [48]:
w2v_file = open("W2V_150.txt", encoding='utf8')
antonym_file = open("Antonym_vietnamese.txt", encoding='utf8')
synonym_file = open("Synonym_vietnamese.txt", encoding='utf8')

In [49]:
antonym_lines = [line for line in antonym_file.readlines()]
synonym_lines = [line for line in synonym_file.readlines()]

In [51]:
words = [line for line in w2v_file.readlines()]
num_of_words = words[0]
dims = words[1]
words = words[2:]

In [52]:
arr_words = []
embedding = {}

In [53]:
for idx, w in enumerate(words):
    arr_words = words[idx].split(' ')
    word = arr_words[0]
    weight = []
    for i in arr_words[2:len(arr_words)-1]:
        weight.append(float(i))
    embedding[word] = np.array(weight)

In [54]:
antonym_word1 = []
antonym_word2 = []
synonym_word1 = []
synonym_word2 = []

for idx, w in enumerate(antonym_lines):
    arr_words = antonym_lines[idx].split(' ')
    antonym_word1.append(arr_words[0])
    antonym_word2.append(arr_words[1].split('\n')[0])

for idx, w in enumerate(synonym_lines):
    arr_words = synonym_lines[idx].split(' ')
    synonym_word1.append(arr_words[0])
    synonym_word2.append(arr_words[1].split('\n')[0])

In [70]:
word_not_in_w2v = []
for w in antonym_word1:
    if w not in dicts.keys():
        word_not_in_w2v.append(w)
for w in antonym_word2:
    if w not in dicts.keys():
        word_not_in_w2v.append(w)
for w in synonym_word1:
    if w not in dicts.keys():
        word_not_in_w2v.append(w)
for w in synonym_word2:
    if w not in dicts.keys():
        word_not_in_w2v.append(w)

In [71]:
word_not_in_w2v = list(dict.fromkeys(word_not_in_w2v))
len(word_not_in_w2v)

4633

In [88]:
missing_words_dict = {}
for w in word_not_in_w2v:
    missing_words_dict[w] = np.random.normal(0, 1, 150).tolist()

In [268]:
dicts.update(missing_words_dict)
len(dicts.keys())

81654

## Train model MLP

In [253]:
X_train = []
y_train = []
for i in range(0, len(synonym_word1)):
    embed_1 = np.asarray(dicts[synonym_word1[i]]).squeeze()
    embed_2 = np.asarray(dicts[synonym_word2[i]]).squeeze()
    sub_embed = embed_1 - embed_2
    X_train.append(np.concatenate((embed_1, embed_2, sub_embed), axis=None))
    y_train.append(1)
for i in range(0, len(antonym_word1)):
    embed_1 = np.asarray(dicts[antonym_word1[i]]).squeeze()
    embed_2 = np.asarray(dicts[antonym_word2[i]]).squeeze()
    sub_embed = embed_1 - embed_2
    X_train.append(np.concatenate((embed_1, embed_2, sub_embed), axis=None))
    y_train.append(0)


In [266]:
X_train = np.asarray(X_train)
X_train.shape

(13562, 450)

In [267]:
y_train = np.asarray(y_train)
y_train.shape

(13562,)

In [258]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam',
                    alpha=1e-5,
                    hidden_layer_sizes=(5, 2), 
                    max_iter=200,
                    random_state=42,
                    verbose=True,).fit(X_train, y_train)

Iteration 1, loss = 0.71573229
Iteration 2, loss = 0.51764974
Iteration 3, loss = 0.44435896
Iteration 4, loss = 0.40507172
Iteration 5, loss = 0.37533956
Iteration 6, loss = 0.34973117
Iteration 7, loss = 0.32832721
Iteration 8, loss = 0.30496261
Iteration 9, loss = 0.28167477
Iteration 10, loss = 0.26279359
Iteration 11, loss = 0.24784583
Iteration 12, loss = 0.23586708
Iteration 13, loss = 0.22513587
Iteration 14, loss = 0.21567851
Iteration 15, loss = 0.20777276
Iteration 16, loss = 0.20033595
Iteration 17, loss = 0.19364220
Iteration 18, loss = 0.18756651
Iteration 19, loss = 0.18211076
Iteration 20, loss = 0.17725168
Iteration 21, loss = 0.17296484
Iteration 22, loss = 0.16864524
Iteration 23, loss = 0.16479917
Iteration 24, loss = 0.16126262
Iteration 25, loss = 0.15840742
Iteration 26, loss = 0.15518358
Iteration 27, loss = 0.15212204
Iteration 28, loss = 0.14949583
Iteration 29, loss = 0.14693924
Iteration 30, loss = 0.14563484
Iteration 31, loss = 0.14316777
Iteration 32, los

/home/longht/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [259]:
n_test = pd.read_csv("400_noun_pairs.txt", sep='\t') #400
v_test = pd.read_csv('400_verb_pairs.txt',sep='\t')  #400
a_test = pd.read_csv('600_adj_pairs.txt', sep='\t')  #600
df_test = pd.concat([n_test, v_test, a_test], ignore_index=True)

In [260]:
ant_test = df_test.loc[df_test['Relation']=='ANT'].reset_index() #700x4
syn_test = df_test.loc[df_test['Relation']=='SYN'].reset_index() #700x4
ant_test

,index,Word1,Word2,Relation
0,0,khoái_lạc,nỗi_đau,ANT
1,1,yếu_kém,sức_mạnh,ANT
2,2,thanh_danh,ô_nhục,ANT
3,5,ba,me,ANT
4,6,ngày,ban_đêm,ANT
...,...,...,...,...
695,1395,ấm_áp,lạnh_lẽo,ANT
696,1396,mập,ngẳng,ANT
697,1397,chóng,lâu,ANT
698,1398,chậm,sớm,ANT


In [261]:
X_test = []
y_test = []
for i in range(0, 700):
    embed_1 = np.asarray(dicts[syn_test['Word1'][i]]).squeeze()
    embed_2 = np.asarray(dicts[syn_test['Word2'][i]]).squeeze()
    sub_embed = embed_1 - embed_2
    X_test.append(np.concatenate((embed_1, embed_2, sub_embed), axis=None))
    y_test.append(1)
for i in range(0, 700):
    embed_1 = np.asarray(dicts[ant_test['Word1'][i]]).squeeze()
    embed_2 = np.asarray(dicts[ant_test['Word2'][i]]).squeeze()
    sub_embed = embed_1 - embed_2
    X_test.append(np.concatenate((embed_1, embed_2, sub_embed), axis=None))
    y_test.append(0)

In [262]:
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

In [263]:
from sklearn.metrics import classification_report
y_pred = clf.predict(X_test)
target_names = ['ANT', 'SYN']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

         ANT       1.00      0.86      0.93       700
         SYN       0.88      1.00      0.94       700

    accuracy                           0.93      1400
   macro avg       0.94      0.93      0.93      1400
weighted avg       0.94      0.93      0.93      1400

